In [60]:
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
# PyTorch related imports would go here
import numpy as np

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
from us import states
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import re

In [62]:
state_names = [state.name.lower() for state in states.STATES]
state_abbrs = [state.abbr for state in states.STATES]
city_df = pd.read_csv('PM25-Speciated/worldcities.csv')
city_names = city_df['city_ascii'].str.lower().tolist()  # First column
country_names = city_df['country'].str.lower().tolist()  # First column

In [63]:
stop_words = TfidfVectorizer(stop_words='english').get_stop_words()
stop_words = list(list(stop_words)+ city_names + country_names + state_names + state_abbrs)

# Original Data

In [64]:
DATASET_NAME = "SpeciateV5.3_PM_All.csv"

df = pd.read_csv('PM25-Speciated/specviate_v5_3datasets/'+DATASET_NAME, encoding='ISO-8859-1')

In [65]:
df

,Unnamed: 0,PROFILE_CODE,WEIGHT_PERCENT,UNCERTAINTY_PERCENT,SPECIES_NAME,SYMBOL,PROFILE_NAME,PROFILE_TYPE,TOTAL,REGION,CATEGORY_LEVEL_1_Generation_Mechanism,CATEGORY_LEVEL_2_Sector_Equipment,CATEGORY_LEVEL_3_.Fuel_Product
0,291,0000010,0.1140,0.089000,Tin,Sn,Overall Composite,PM,55.476002,NaN,Miscellaneous,Miscellaneous,Miscellaneous
1,292,0000010,2.6320,1.464000,Sulfur,S,Overall Composite,PM,55.476002,NaN,Miscellaneous,Miscellaneous,Miscellaneous
2,293,0000010,5.0460,3.574000,Sulfate,SO4=,Overall Composite,PM,55.476002,NaN,Miscellaneous,Miscellaneous,Miscellaneous
3,295,0000010,1.3770,1.535000,Sodium,Na,Overall Composite,PM,55.476002,NaN,Miscellaneous,Miscellaneous,Miscellaneous
4,297,0000010,0.0250,0.418000,Selenium,Se,Overall Composite,PM,55.476002,NaN,Miscellaneous,Miscellaneous,Miscellaneous
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76500,331373,95780,0.0000,0.123300,Nitrate,NO3-,Paved Road Dust,PM,55.612400,"Lake Tahoe, Nevada",Dust,Road,Paved
76501,331375,95780,0.0000,0.009700,Mercury,Hg,Paved Road Dust,PM,55.612400,"Lake Tahoe, Nevada",Dust,Road,Paved
76502,331376,95780,0.0269,0.012666,Lead,Pb,Paved Road Dust,PM,55.612400,"Lake Tahoe, Nevada",Dust,Road,Paved
76503,331377,95780,0.1509,0.419457,Magnesium,Mg,Paved Road Dust,PM,55.612400,"Lake Tahoe, Nevada",Dust,Road,Paved


In [66]:
df = df.pivot_table(index=['PROFILE_CODE', 'PROFILE_NAME'], columns='SPECIES_NAME', values='WEIGHT_PERCENT', aggfunc='first').reset_index()
# Fill NaN values with 0 (or any other value you prefer)
df = df.fillna(0)

In [67]:
df

SPECIES_NAME,PROFILE_CODE,PROFILE_NAME,Aluminum,Ammonium,Antimony,Arsenic,Bromine,Cadmium,Calcium,Calcium ion,...,Silica,Silicon,Sodium,Sodium ion,Sulfate,Sulfur,Tin,Vanadium,Zinc,Zirconium
0,0000010,Overall Composite,3.81900,0.023000,0.8090,1.410000,0.0,0.385000,2.399000,0.0,...,0.0,7.736000,1.377000,0.0000,5.046000,2.632000,0.114000,0.058000,2.218000,0.002000
1,000002.5,Overall Composite,4.44300,0.022000,0.7650,1.476000,0.0,0.421000,2.614000,0.0,...,0.0,9.491000,1.652000,0.0000,5.514000,2.972000,0.117000,0.061000,2.233000,0.002000
2,0000030,Overall Composite,4.02400,0.022000,0.8280,1.175000,0.0,0.377000,2.945000,0.0,...,0.0,8.673000,1.407000,0.0000,6.169000,2.876000,0.162000,0.062000,2.392000,0.003000
3,00000C,Overall Composite,4.07900,0.030000,0.8060,1.118000,0.0,0.285000,2.804000,0.0,...,0.0,8.933000,1.577000,0.0000,5.030000,2.457000,0.103000,0.059000,2.235000,0.004000
4,1120110,Coal-Fired Power Plant,14.84300,0.000000,0.0000,0.055000,0.0,0.005000,1.393000,0.0,...,0.0,23.290000,0.000000,0.0000,0.000000,1.803000,0.006000,0.072000,0.055000,0.053000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3389,95776,Residential Wood Combustion - Composite of fir...,0.01617,0.154896,0.0013,0.000492,0.0,0.002611,0.094026,0.0,...,0.0,0.061561,0.023333,0.0457,0.217874,0.101063,0.000713,0.000236,0.038925,0.000695
3390,95777,Paved Road Dust,10.31170,0.098100,0.0000,0.000000,0.0,0.000000,3.964900,0.0,...,0.0,30.365200,0.140500,0.1365,0.809900,0.591800,0.000000,0.006200,0.095100,0.034500
3391,95778,Paved Road Dust,5.01290,0.168000,0.0094,0.000000,0.0,0.000000,2.393000,0.0,...,0.0,15.603500,0.339700,0.1454,0.463600,0.646400,0.007000,0.000000,0.088800,0.033300
3392,95779,Paved Road Dust,8.89020,0.068400,0.0000,0.000200,0.0,0.000000,3.314800,0.0,...,0.0,29.196600,0.153500,0.0943,0.200500,0.369700,0.001100,0.008900,0.067600,0.019000


In [68]:
def clean_text(text):
    # Remove numbers and special characters
    text = re.sub(r'[^A-Za-z\s]', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

In [69]:
# df['name'] = df['name'].apply(clean_text)

In [70]:
df

SPECIES_NAME,PROFILE_CODE,PROFILE_NAME,Aluminum,Ammonium,Antimony,Arsenic,Bromine,Cadmium,Calcium,Calcium ion,...,Silica,Silicon,Sodium,Sodium ion,Sulfate,Sulfur,Tin,Vanadium,Zinc,Zirconium
0,0000010,Overall Composite,3.81900,0.023000,0.8090,1.410000,0.0,0.385000,2.399000,0.0,...,0.0,7.736000,1.377000,0.0000,5.046000,2.632000,0.114000,0.058000,2.218000,0.002000
1,000002.5,Overall Composite,4.44300,0.022000,0.7650,1.476000,0.0,0.421000,2.614000,0.0,...,0.0,9.491000,1.652000,0.0000,5.514000,2.972000,0.117000,0.061000,2.233000,0.002000
2,0000030,Overall Composite,4.02400,0.022000,0.8280,1.175000,0.0,0.377000,2.945000,0.0,...,0.0,8.673000,1.407000,0.0000,6.169000,2.876000,0.162000,0.062000,2.392000,0.003000
3,00000C,Overall Composite,4.07900,0.030000,0.8060,1.118000,0.0,0.285000,2.804000,0.0,...,0.0,8.933000,1.577000,0.0000,5.030000,2.457000,0.103000,0.059000,2.235000,0.004000
4,1120110,Coal-Fired Power Plant,14.84300,0.000000,0.0000,0.055000,0.0,0.005000,1.393000,0.0,...,0.0,23.290000,0.000000,0.0000,0.000000,1.803000,0.006000,0.072000,0.055000,0.053000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3389,95776,Residential Wood Combustion - Composite of fir...,0.01617,0.154896,0.0013,0.000492,0.0,0.002611,0.094026,0.0,...,0.0,0.061561,0.023333,0.0457,0.217874,0.101063,0.000713,0.000236,0.038925,0.000695
3390,95777,Paved Road Dust,10.31170,0.098100,0.0000,0.000000,0.0,0.000000,3.964900,0.0,...,0.0,30.365200,0.140500,0.1365,0.809900,0.591800,0.000000,0.006200,0.095100,0.034500
3391,95778,Paved Road Dust,5.01290,0.168000,0.0094,0.000000,0.0,0.000000,2.393000,0.0,...,0.0,15.603500,0.339700,0.1454,0.463600,0.646400,0.007000,0.000000,0.088800,0.033300
3392,95779,Paved Road Dust,8.89020,0.068400,0.0000,0.000200,0.0,0.000000,3.314800,0.0,...,0.0,29.196600,0.153500,0.0943,0.200500,0.369700,0.001100,0.008900,0.067600,0.019000


In [71]:
# numeric_cols = df.select_dtypes(include=[np.number]).columns
# text_cols = df.select_dtypes(include=[object]).columns

# print(text_cols)

# agg_funcs = {col: 'mean' for col in numeric_cols}
# agg_funcs.update({col: lambda x: np.random.choice(x) for col in text_cols if col != 'name'})

# pivot_df = df.groupby('name').agg(agg_funcs).reset_index()

In [72]:
# cols = list(pivot_df.columns)
# cols.insert(0, cols.pop(cols.index('code')))
# pivot_df = pivot_df[cols]
# pivot_df

In [73]:
pivot_df = df
pivot_df

SPECIES_NAME,PROFILE_CODE,PROFILE_NAME,Aluminum,Ammonium,Antimony,Arsenic,Bromine,Cadmium,Calcium,Calcium ion,...,Silica,Silicon,Sodium,Sodium ion,Sulfate,Sulfur,Tin,Vanadium,Zinc,Zirconium
0,0000010,Overall Composite,3.81900,0.023000,0.8090,1.410000,0.0,0.385000,2.399000,0.0,...,0.0,7.736000,1.377000,0.0000,5.046000,2.632000,0.114000,0.058000,2.218000,0.002000
1,000002.5,Overall Composite,4.44300,0.022000,0.7650,1.476000,0.0,0.421000,2.614000,0.0,...,0.0,9.491000,1.652000,0.0000,5.514000,2.972000,0.117000,0.061000,2.233000,0.002000
2,0000030,Overall Composite,4.02400,0.022000,0.8280,1.175000,0.0,0.377000,2.945000,0.0,...,0.0,8.673000,1.407000,0.0000,6.169000,2.876000,0.162000,0.062000,2.392000,0.003000
3,00000C,Overall Composite,4.07900,0.030000,0.8060,1.118000,0.0,0.285000,2.804000,0.0,...,0.0,8.933000,1.577000,0.0000,5.030000,2.457000,0.103000,0.059000,2.235000,0.004000
4,1120110,Coal-Fired Power Plant,14.84300,0.000000,0.0000,0.055000,0.0,0.005000,1.393000,0.0,...,0.0,23.290000,0.000000,0.0000,0.000000,1.803000,0.006000,0.072000,0.055000,0.053000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3389,95776,Residential Wood Combustion - Composite of fir...,0.01617,0.154896,0.0013,0.000492,0.0,0.002611,0.094026,0.0,...,0.0,0.061561,0.023333,0.0457,0.217874,0.101063,0.000713,0.000236,0.038925,0.000695
3390,95777,Paved Road Dust,10.31170,0.098100,0.0000,0.000000,0.0,0.000000,3.964900,0.0,...,0.0,30.365200,0.140500,0.1365,0.809900,0.591800,0.000000,0.006200,0.095100,0.034500
3391,95778,Paved Road Dust,5.01290,0.168000,0.0094,0.000000,0.0,0.000000,2.393000,0.0,...,0.0,15.603500,0.339700,0.1454,0.463600,0.646400,0.007000,0.000000,0.088800,0.033300
3392,95779,Paved Road Dust,8.89020,0.068400,0.0000,0.000200,0.0,0.000000,3.314800,0.0,...,0.0,29.196600,0.153500,0.0943,0.200500,0.369700,0.001100,0.008900,0.067600,0.019000


In [74]:
pivot_df.iloc[:,2:47].head()

SPECIES_NAME,Aluminum,Ammonium,Antimony,Arsenic,Bromine,Cadmium,Calcium,Calcium ion,Chloride ion,Chlorine,...,Silica,Silicon,Sodium,Sodium ion,Sulfate,Sulfur,Tin,Vanadium,Zinc,Zirconium
0,3.819,0.023,0.809,1.410,0.0,0.385,2.399,0.0,0.0,0.0,...,0.0,7.736,1.377,0.0,5.046,2.632,0.114,0.058,2.218,0.002
1,4.443,0.022,0.765,1.476,0.0,0.421,2.614,0.0,0.0,0.0,...,0.0,9.491,1.652,0.0,5.514,2.972,0.117,0.061,2.233,0.002
2,4.024,0.022,0.828,1.175,0.0,0.377,2.945,0.0,0.0,0.0,...,0.0,8.673,1.407,0.0,6.169,2.876,0.162,0.062,2.392,0.003
3,4.079,0.030,0.806,1.118,0.0,0.285,2.804,0.0,0.0,0.0,...,0.0,8.933,1.577,0.0,5.030,2.457,0.103,0.059,2.235,0.004
4,14.843,0.000,0.000,0.055,0.0,0.005,1.393,0.0,0.0,0.0,...,0.0,23.290,0.000,0.0,0.000,1.803,0.006,0.072,0.055,0.053


# Clustering based on numeric values

In [75]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from collections import Counter

In [76]:
def assign_cluster_name(cluster):
    names = pivot_df[pivot_df['speciated_cluster'] == cluster]['PROFILE_NAME']
    most_common_name = Counter(names).most_common(1)[0][0]
    return most_common_name

In [77]:
X = pivot_df.iloc[:, 2:45]
#scaler = StandardScaler()
# scaler = MinMaxScaler()
# X = scaler.fit_transform(X)

In [78]:
X

SPECIES_NAME,Aluminum,Ammonium,Antimony,Arsenic,Bromine,Cadmium,Calcium,Calcium ion,Chloride ion,Chlorine,...,Pyrolyzed organic carbon,Selenium,Silica,Silicon,Sodium,Sodium ion,Sulfate,Sulfur,Tin,Vanadium
0,3.81900,0.023000,0.8090,1.410000,0.0,0.385000,2.399000,0.0,0.000000,0.0,...,0.000000,0.025000,0.0,7.736000,1.377000,0.0000,5.046000,2.632000,0.114000,0.058000
1,4.44300,0.022000,0.7650,1.476000,0.0,0.421000,2.614000,0.0,0.000000,0.0,...,0.000000,0.025000,0.0,9.491000,1.652000,0.0000,5.514000,2.972000,0.117000,0.061000
2,4.02400,0.022000,0.8280,1.175000,0.0,0.377000,2.945000,0.0,0.000000,0.0,...,0.000000,0.029000,0.0,8.673000,1.407000,0.0000,6.169000,2.876000,0.162000,0.062000
3,4.07900,0.030000,0.8060,1.118000,0.0,0.285000,2.804000,0.0,0.000000,0.0,...,0.000000,0.023000,0.0,8.933000,1.577000,0.0000,5.030000,2.457000,0.103000,0.059000
4,14.84300,0.000000,0.0000,0.055000,0.0,0.005000,1.393000,0.0,0.000000,0.0,...,0.000000,0.018000,0.0,23.290000,0.000000,0.0000,0.000000,1.803000,0.006000,0.072000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3389,0.01617,0.154896,0.0013,0.000492,0.0,0.002611,0.094026,0.0,0.120587,0.0,...,1.983234,0.000199,0.0,0.061561,0.023333,0.0457,0.217874,0.101063,0.000713,0.000236
3390,10.31170,0.098100,0.0000,0.000000,0.0,0.000000,3.964900,0.0,0.106100,0.0,...,3.957000,0.000000,0.0,30.365200,0.140500,0.1365,0.809900,0.591800,0.000000,0.006200
3391,5.01290,0.168000,0.0094,0.000000,0.0,0.000000,2.393000,0.0,0.144400,0.0,...,2.190400,0.000000,0.0,15.603500,0.339700,0.1454,0.463600,0.646400,0.007000,0.000000
3392,8.89020,0.068400,0.0000,0.000200,0.0,0.000000,3.314800,0.0,0.066100,0.0,...,3.889100,0.000100,0.0,29.196600,0.153500,0.0943,0.200500,0.369700,0.001100,0.008900


In [79]:
model = NMF(n_components=32, init='random', random_state=0, max_iter=1000)
W = model.fit_transform(X)
H = model.components_

# What features to use for clustering

In [80]:
# Assuming 'target' is the category column, and 'name' is the label
X = pivot_df.iloc[:, 2:45]
#X = W

In [81]:
n_clusters = 16
kmeans = KMeans(n_clusters=n_clusters, random_state=42,n_init='auto')
pivot_df['speciated_cluster'] = kmeans.fit_predict(X)

# Create a new column for cluster names
pivot_df['speciated_cluster_name'] = pivot_df['speciated_cluster'].apply(assign_cluster_name)

pivot_df.head()

SPECIES_NAME,PROFILE_CODE,PROFILE_NAME,Aluminum,Ammonium,Antimony,Arsenic,Bromine,Cadmium,Calcium,Calcium ion,...,Sodium,Sodium ion,Sulfate,Sulfur,Tin,Vanadium,Zinc,Zirconium,speciated_cluster,speciated_cluster_name
0,0000010,Overall Composite,3.819,0.023,0.809,1.410,0.0,0.385,2.399,0.0,...,1.377,0.0,5.046,2.632,0.114,0.058,2.218,0.002,1,Motor Vehicle Exhaust
1,000002.5,Overall Composite,4.443,0.022,0.765,1.476,0.0,0.421,2.614,0.0,...,1.652,0.0,5.514,2.972,0.117,0.061,2.233,0.002,0,Paved Road Dust
2,0000030,Overall Composite,4.024,0.022,0.828,1.175,0.0,0.377,2.945,0.0,...,1.407,0.0,6.169,2.876,0.162,0.062,2.392,0.003,1,Motor Vehicle Exhaust
3,00000C,Overall Composite,4.079,0.030,0.806,1.118,0.0,0.285,2.804,0.0,...,1.577,0.0,5.030,2.457,0.103,0.059,2.235,0.004,1,Motor Vehicle Exhaust
4,1120110,Coal-Fired Power Plant,14.843,0.000,0.000,0.055,0.0,0.005,1.393,0.0,...,0.000,0.0,0.000,1.803,0.006,0.072,0.055,0.053,4,Agriculture Soil


In [82]:
for cluster_num in range(n_clusters):
    print(f"\nCluster {cluster_num}:")
    cluster_texts = pivot_df[pivot_df['speciated_cluster'] == cluster_num]['PROFILE_NAME']
    for text in cluster_texts:
        print(f" - {text}")


Cluster 0:
 - Overall Composite
 - Coal-Fired Power Plant
 - Coal-Fired Power Plant
 - Coal-Fired Power Plant
 - Coal-Fired Power Plant
 - Coal-Fired Power Plant
 - Coal-Fired Power Plant
 - Coal-Fired Power Plant
 - Coal-Fired Power Plant
 - Coal-Fired Power Plant
 - Coal-Fired Power Plant
 - Coal-Fired Power Plant
 - Coal-Fired Power Plant
 - Coal-Fired Power Plant
 - Coal-Fired Power Plant/esp Composite
 - Coal-Fired Power Plant/esp Composite
 - Coal- And Refuse Derived Fuel (RDF)-Fired Power Plant
 - Coal- And Refuse Derived Fuel (RDF)-Fired Power Plant
 - Coal- And Refuse Derived Fuel (RDF)-Fired Power Plant
 - Coal- And Refuse Derived Fuel (RDF)-Fired Power Plant
 - Oil-Fired Power Plant
 - External Combustion Boiler - Coal-Slurry Fired
 - External Combustion Boiler - Coal-Slurry Fired
 - External Combustion Boiler - Coal-Slurry Fired
 - Municipal Incinerator (Philadelphia)
 - Sewage Sludge Incineration
 - Sewage Sludge Incineration
 - Sewage Sludge Incineration
 - Sewage Sludge

# Hierarchical Clustering

In [83]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from collections import Counter
plt.rcParams['font.family'] = 'Sans'

In [84]:
# Assuming 'target' is the category column, and 'name' is the label
# X = pivot_df.iloc[:, 2:47]

# Perform Agglomerative Clustering with more branches (e.g., 5 clusters)
max_clusters = 10
#clustering = AgglomerativeClustering(n_clusters=max_clusters, linkage='ward')
clustering = AgglomerativeClustering(n_clusters=max_clusters, linkage='ward', metric='euclidean')
pivot_df['cluster'] = clustering.fit_predict(X)

# Assign majority name within each cluster
def get_majority_label(cluster, names):
    return Counter(names).most_common(1)[0][0]

cluster_majority_labels = {
    cluster: get_majority_label(cluster, pivot_df[pivot_df['cluster'] == cluster]['PROFILE_NAME'])
    for cluster in range(max_clusters)
}

# Replace cluster numbers with majority names
pivot_df['cluster_label'] = pivot_df['cluster'].map(cluster_majority_labels)

# Plot the clusters (using seaborn scatter plot for visualization)
plt.figure(figsize=(10, 7))

# Initialize NMF and fit to the data
# model = NMF(n_components=2, init='random', random_state=0)
# W = model.fit_transform(X)
# H = model.components_


sns.scatterplot(data=pivot_df, x=W[:, 0], y=W[:, 1], hue='cluster_label', palette='tab10', s=100)
plt.title(f'Clustering with {max_clusters} Branches')
plt.show()


## Dendogram Hierarchy

In [85]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from collections import Counter
from scipy.cluster.hierarchy import dendrogram, linkage

# Assuming 'target' is the category column, and 'name' is the label
# X = pivot_df.iloc[:, 2:47]

# Perform Agglomerative Clustering with more branches (e.g., 5 clusters)
max_clusters = 24
clustering = AgglomerativeClustering(n_clusters=max_clusters, linkage='ward')
pivot_df['cluster'] = clustering.fit_predict(X)

# Assign majority name within each cluster
def get_majority_label(cluster, names):
    return Counter(names).most_common(1)[0][0]

cluster_majority_labels = {}
for cluster in range(max_clusters):
    cluster_names = pivot_df[pivot_df['cluster'] == cluster]['PROFILE_NAME']
    majority_label = get_majority_label(cluster, cluster_names)
    cluster_majority_labels[cluster] = majority_label
    # Print cluster representative name and count of items in cluster
    print(f"Cluster {cluster}: Representative Name = {majority_label}, Number of Items = {len(cluster_names)}")

# Replace cluster numbers with majority names
pivot_df['cluster_label'] = pivot_df['cluster'].map(cluster_majority_labels)

# Generate the linkage matrix
Z = linkage(X, method='ward')

# Plot the circular dendrogram
plt.figure(figsize=(10, 7))
dendrogram(Z, truncate_mode='lastp', p=max_clusters, labels=pivot_df['cluster_label'].values, leaf_font_size=10, orientation='right')
plt.title(f'Circular Dendrogram of Hierarchical Clustering with {max_clusters} Branches')
plt.show()


Cluster 0: Representative Name = Light Duty Vehicles - Leaded, Number of Items = 83
Cluster 1: Representative Name = Paved Road Dust, Number of Items = 653
Cluster 2: Representative Name = Paved Road Dust, Number of Items = 211
Cluster 3: Representative Name = Motor Vehicle Exhaust, Number of Items = 1384
Cluster 4: Representative Name = Industrial Soil, Number of Items = 57
Cluster 5: Representative Name = Cement Kiln, Number of Items = 88
Cluster 6: Representative Name = Oil-Fired Power Plant, Number of Items = 111
Cluster 7: Representative Name = Cooking, Number of Items = 40
Cluster 8: Representative Name = Oil Refinery, Number of Items = 16
Cluster 9: Representative Name = Diesel Exhaust, Number of Items = 64
Cluster 10: Representative Name = Motor Vehicle Exhaust, Number of Items = 20
Cluster 11: Representative Name = Kraft Recovery Furnace, Number of Items = 17
Cluster 12: Representative Name = Primary Copper Smelter, Number of Items = 12
Cluster 13: Representative Name = Reside

### Dendogram of all points

In [86]:
plt.figure(figsize=(10, 100))
dendrogram(Z, labels=pivot_df['PROFILE_NAME'].values, leaf_font_size=10, orientation='right')
plt.title(f'Circular Dendrogram of Hierarchical Clustering with {max_clusters} Branches')
plt.show()


## Print Top k representatives within each clusters

In [87]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from collections import Counter
from scipy.cluster.hierarchy import dendrogram, linkage

# Assuming 'target' is the category column, and 'name' is the label
# X = pivot_df.iloc[:, 2:47]

# Perform Agglomerative Clustering with more branches (e.g., 20 clusters)
max_clusters = 10
clustering = AgglomerativeClustering(n_clusters=max_clusters, linkage='ward')
pivot_df['cluster'] = clustering.fit_predict(X)

# Modify the get_majority_label function to return top k labels with their counts
def get_top_k_labels(cluster, names, k=3):
    counter = Counter(names).most_common(k)
    return [(label, count) for label, count in counter]

cluster_majority_labels = {}
k = 1  # Choose the number of majority names to show

for cluster in range(max_clusters):
    cluster_names = pivot_df[pivot_df['cluster'] == cluster]['PROFILE_NAME']
    top_k_labels = get_top_k_labels(cluster, cluster_names, k)
    
    # Combine top k labels into a string format 'Label1 (Count1), Label2 (Count2), ...'
    majority_label_str = ', '.join([f"{label} ({count})" for label, count in top_k_labels])
    
    cluster_majority_labels[cluster] = majority_label_str
    
    # Print cluster representative names and counts
    print(f"Cluster {cluster}: Representative Names = {majority_label_str}, Number of Items = {len(cluster_names)}")
    print("")

# Replace cluster numbers with majority names and their counts
pivot_df['cluster_label'] = pivot_df['cluster'].map(cluster_majority_labels)

# Generate the linkage matrix
Z = linkage(X, method='ward')

# Plot the circular dendrogram with updated labels
plt.figure(figsize=(10, 7))
dendrogram(Z, truncate_mode='lastp', p= max_clusters, labels=pivot_df['cluster_label'].values, leaf_font_size=10, orientation='right')
plt.title(f'Circular Dendrogram of Hierarchical Clustering with {max_clusters} Branches')
plt.show()


Cluster 0: Representative Names = Motor Vehicle Exhaust (130), Number of Items = 1514

Cluster 1: Representative Names = Primary Copper Converter - Secondary Hood (7), Number of Items = 120

Cluster 2: Representative Names = Oil-Fired Power Plant (24), Number of Items = 144

Cluster 3: Representative Names = Motor Vehicle Exhaust (44), Number of Items = 341

Cluster 4: Representative Names = Diesel Exhaust (36), Number of Items = 103

Cluster 5: Representative Names = Salting Material (1), Number of Items = 1

Cluster 6: Representative Names = Paved Road Dust (28), Number of Items = 211

Cluster 7: Representative Names = Motor Vehicle Exhaust (1), Number of Items = 1

Cluster 8: Representative Names = Agriculture Soil (104), Number of Items = 902

Cluster 9: Representative Names = Industrial Soil (7), Number of Items = 57



## k-NN Graph Construction

In [88]:
from sklearn.neighbors import NearestNeighbors
import networkx as nx

In [89]:
# Compute KNN

K = 3
knn = NearestNeighbors(n_neighbors=K)  # You can change the number of neighbors
knn.fit(X)
distances, indices = knn.kneighbors(X)

# Create a graph
G = nx.Graph()

# Add nodes with PROFILE_NAME as labels
for i, profile_name in enumerate(df['PROFILE_NAME']):
    G.add_node(i, label=profile_name)

# Add edges based on KNN
for i, neighbors in enumerate(indices):
    for neighbor in neighbors:
        if i != neighbor:  # Avoid self-loops
            G.add_edge(i, neighbor)

In [90]:
import matplotlib
matplotlib.use('Agg')  # Use a non-interactive backend

In [91]:
# pos = nx.spring_layout(G)  # Layout for visualization
# labels = nx.get_node_attributes(G, 'label')

# plt.figure(figsize=(50, 50))
# nx.draw(G, pos, with_labels=True, labels=labels, node_size=100, node_color='skyblue', font_size=10, font_color='black', edge_color='gray')
# plt.title('K-Nearest Neighbor Graph')
# plt.show()

In [92]:
# nx.write_graphml(G, 'knn_graph.graphml')


labels=[]
y = [i for i in pivot_df['PROFILE_NAME']]
labels = dict(zip(range(len(y)), y))

# print(labels)

nx.set_node_attributes(G, labels, 'labels')
print("Writing gephi....")

nx.write_gexf(G, DATASET_NAME+"KNN"+str(K)+'.gexf')
print("Done....")

Writing gephi....
Done....


## Data preprocessing

In [93]:
category_for_classification = 'speciated_cluster_name' 
#'assigned_profile', #'speciated_cluster_name'

In [94]:
# # Step 1: Read the CSV into a pandas dataframe

# filename = 'PM25-Speciated/IMPROVEDataset/usepa_final_with_assigned_profile.csv'
# df = pd.read_csv(filename)

In [95]:
# df

In [96]:
# Create a dictionary to map categories to numerical values
category_mapping = {category: index for index, category in enumerate(pivot_df[category_for_classification].unique())}

In [97]:
# Replace string categories with numerical values
pivot_df['final_profile'] = pivot_df[category_for_classification].map(category_mapping)

# The updated DataFrame and the mapping dictionary
print(category_mapping)
pivot_df

{'Motor Vehicle Exhaust': 0, 'Paved Road Dust': 1, 'Agriculture Soil': 2, 'Oil-Fired Power Plant': 3, 'Coal Combustion': 4, 'Primary Copper Converter - Secondary Hood': 5, 'Ammonium Nitrate - Prill Tower': 6, 'Iron and Steel Manufacturing': 7, 'Primary Copper Smelter': 8, 'Residential Wood Burning': 9, 'Cooking': 10, 'Diesel Exhaust': 11, 'Salting Material': 12}


SPECIES_NAME,PROFILE_CODE,PROFILE_NAME,Aluminum,Ammonium,Antimony,Arsenic,Bromine,Cadmium,Calcium,Calcium ion,...,Sulfur,Tin,Vanadium,Zinc,Zirconium,speciated_cluster,speciated_cluster_name,cluster,cluster_label,final_profile
0,0000010,Overall Composite,3.81900,0.023000,0.8090,1.410000,0.0,0.385000,2.399000,0.0,...,2.632000,0.114000,0.058000,2.218000,0.002000,1,Motor Vehicle Exhaust,0,Motor Vehicle Exhaust (130),0
1,000002.5,Overall Composite,4.44300,0.022000,0.7650,1.476000,0.0,0.421000,2.614000,0.0,...,2.972000,0.117000,0.061000,2.233000,0.002000,0,Paved Road Dust,0,Motor Vehicle Exhaust (130),1
2,0000030,Overall Composite,4.02400,0.022000,0.8280,1.175000,0.0,0.377000,2.945000,0.0,...,2.876000,0.162000,0.062000,2.392000,0.003000,1,Motor Vehicle Exhaust,0,Motor Vehicle Exhaust (130),0
3,00000C,Overall Composite,4.07900,0.030000,0.8060,1.118000,0.0,0.285000,2.804000,0.0,...,2.457000,0.103000,0.059000,2.235000,0.004000,1,Motor Vehicle Exhaust,0,Motor Vehicle Exhaust (130),0
4,1120110,Coal-Fired Power Plant,14.84300,0.000000,0.0000,0.055000,0.0,0.005000,1.393000,0.0,...,1.803000,0.006000,0.072000,0.055000,0.053000,4,Agriculture Soil,8,Agriculture Soil (104),2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3389,95776,Residential Wood Combustion - Composite of fir...,0.01617,0.154896,0.0013,0.000492,0.0,0.002611,0.094026,0.0,...,0.101063,0.000713,0.000236,0.038925,0.000695,5,Cooking,0,Motor Vehicle Exhaust (130),10
3390,95777,Paved Road Dust,10.31170,0.098100,0.0000,0.000000,0.0,0.000000,3.964900,0.0,...,0.591800,0.000000,0.006200,0.095100,0.034500,4,Agriculture Soil,8,Agriculture Soil (104),2
3391,95778,Paved Road Dust,5.01290,0.168000,0.0094,0.000000,0.0,0.000000,2.393000,0.0,...,0.646400,0.007000,0.000000,0.088800,0.033300,0,Paved Road Dust,8,Agriculture Soil (104),1
3392,95779,Paved Road Dust,8.89020,0.068400,0.0000,0.000200,0.0,0.000000,3.314800,0.0,...,0.369700,0.001100,0.008900,0.067600,0.019000,4,Agriculture Soil,8,Agriculture Soil (104),2


In [98]:
len(category_mapping)

13

In [99]:
mapping_category = {value:key for key, value in category_mapping.items()}

In [100]:
# Step 2: Apply Non-negative matrix factorization on the data except last column
# Handle missing values by filling them with the mean of the column
# df.fillna(df.mean(), inplace=True)

In [101]:
# Separate features and target
X = pivot_df.iloc[:, 2:45].values
y = pivot_df['final_profile'].values

In [102]:
print(X.shape)
print(y.shape)

(3394, 43)
(3394,)


## NMF

In [103]:
# # Initialize NMF and fit to the data
# model = NMF(n_components=2, init='random', random_state=0)
# W = model.fit_transform(X)
# H = model.components_

# # Visualize the data
# plt.scatter(W[:, 0], W[:, 1], c=y)
# plt.xlabel('Component 1')
# plt.ylabel('Component 2')
# plt.title('NMF Components')
# plt.colorbar()
# plt.show()

## MLP

In [104]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import random
import numpy as np

In [105]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Example usage
set_seed(42)

In [106]:
# Step 3: Use PyTorch MLP to classify the last column
# Preprocess the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [107]:
X_scaled = torch.FloatTensor(X_scaled)
y = torch.LongTensor(y)

In [108]:
# Assuming X and y are already defined as PyTorch tensors
# X = features tensor
# y = labels tensor

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Create datasets and dataloaders
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [109]:
print(X_train.shape)
print(X_test.shape)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

torch.Size([2715, 43])
torch.Size([679, 43])


In [110]:

# Define a simple MLP model for multi-class classification
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(X_train.shape[1], 256),
            nn.ReLU(),
            nn.Linear(256, len(category_mapping))  # Adjusted for 29 classes
        )
        
    def forward(self, x):
        return self.layers(x)

# Initialize the model
model = MLP()

In [111]:
model

MLP(
  (layers): Sequential(
    (0): Linear(in_features=43, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=13, bias=True)
  )
)

In [112]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification
optimizer = torch.optim.Adam(model.parameters())

In [113]:
def train():
    # Training loop
    for epoch in range(1000):  # number of epochs can be adjusted
        for batch_idx, (data, target) in enumerate(train_loader):
            # Forward pass
            output = model(data)
            loss = criterion(output, target)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if epoch % 50 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")

# train()

In [114]:
def test():
    model.eval()
    test_loss = 0
    all_targets = []
    all_predictions = []

    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += criterion(output, target).item()
            all_targets.extend(target.tolist())
            all_predictions.extend(output.argmax(dim=1).tolist())

    test_loss /= len(test_loader.dataset)
    print(f"Test Loss: {test_loss}")

    # Calculate F1 score
    f1 = f1_score(all_targets, all_predictions, average='weighted')
    print(f"F1 Score: {f1}")
    
    return all_predictions, all_targets
    
pred, target = test()

Test Loss: 0.041433461460343865
F1 Score: 0.2739234739075877


In [115]:
for i, j in zip(pred, target):
    print("Pred:",mapping_category[i],"\n","True:",mapping_category[j])
    print()

Pred: Diesel Exhaust 
 True: Primary Copper Converter - Secondary Hood

Pred: Diesel Exhaust 
 True: Agriculture Soil

Pred: Cooking 
 True: Agriculture Soil

Pred: Cooking 
 True: Paved Road Dust

Pred: Cooking 
 True: Motor Vehicle Exhaust

Pred: Residential Wood Burning 
 True: Diesel Exhaust

Pred: Cooking 
 True: Agriculture Soil

Pred: Residential Wood Burning 
 True: Primary Copper Converter - Secondary Hood

Pred: Agriculture Soil 
 True: Motor Vehicle Exhaust

Pred: Diesel Exhaust 
 True: Agriculture Soil

Pred: Motor Vehicle Exhaust 
 True: Motor Vehicle Exhaust

Pred: Motor Vehicle Exhaust 
 True: Motor Vehicle Exhaust

Pred: Residential Wood Burning 
 True: Motor Vehicle Exhaust

Pred: Motor Vehicle Exhaust 
 True: Motor Vehicle Exhaust

Pred: Diesel Exhaust 
 True: Paved Road Dust

Pred: Diesel Exhaust 
 True: Agriculture Soil

Pred: Diesel Exhaust 
 True: Paved Road Dust

Pred: Motor Vehicle Exhaust 
 True: Motor Vehicle Exhaust

Pred: Oil-Fired Power Plant 
 True: Motor 